# Задание 1

Процедура генерация датасета:
Сгенерируйте датасет с 10000 наблюдений и 1000 колонок (сэмплируйте из разных распеределений) и сформируйте из него таргет на сонове 100 колонок + зашумление (общее или небольшое для каждой колонки - постарайтесь сделать так чтобы шум не сильно влиял на корреляции между предикторами и таргетам). Удостоверьтесь, что в датасете существуют колонки, которые не использовались для таргета, но при этом имеют высокую корреляцию с теми, что использовались (покажите это в коде).

Реализуйте forward stage wise регрессию стандартным образом и с помощью QR разложения наиболее быстрым образом (засекайте время для всех опробованных вариантов). Замерьте качество и процент колонок, которые были правильно найдены.

**Дополнительно**: 
Попробуйте генерировать данные таким образом, чтобы ошибка постепенно ухудшалась. Подсказка: увеличивайте шум, используйте нелинейные функции и комбинации предикторов. Попробуйте оценить bias и variance для forward stage-wise regression.

# Main

In [251]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [252]:
PREDICTOR_CNT = 1_000
TARGET_PREDICTOR_CNT = 100
SAMPLE_CNT = 10_000

In [253]:
class DistributionGenerator:
    @staticmethod
    def generate_random_exponential(n: int = 10000, scale: float | None = None):
        """
        Generate random exponential distribution
        :param n: number of samples
        :param scale: scale parameter of the exponential distribution
        :return: np.array of random samples
        """

        if scale is None:
            scale = np.random.uniform(0.75, 2.0)
        return np.random.exponential(scale=scale, size=n)

    @staticmethod
    def generate_random_uniform(n: int = 10000, low: float | None = None, length: float | None = None):
        """
        Generate random uniform distribution
        :param n: number of samples
        :param low: lower bound of the uniform distribution
        :param length: length of the uniform distribution
        :return: np.array of random samples
        """

        # Kekw: using uniform to generate limits to uniform
        if low is None:
            low = np.random.uniform(-2.0, -1.0)
        if length is None:
            length = np.random.uniform(1.0, 3.0)
        return np.random.uniform(low=low, high=low + length, size=n)

    @staticmethod
    def generate_random_normal(n: int = 10000, loc: float | None = None, scale: float | None = None):
        """
        Generate random normal distribution
        :param n: number of samples
        :param loc: mean of the normal distribution
        :param scale: standard deviation of the normal distribution
        :return: np.array of random samples
        """

        if loc is None:
            loc = np.random.uniform(-1.0, 1.0)
        if scale is None:
            scale = np.random.uniform(0.2, 2.0)
        return np.random.normal(loc=loc, scale=scale, size=n)

    distribution_fn_list = [
        # generate_random_exponential,
        # generate_random_uniform,
        generate_random_normal,
    ]
    @staticmethod
    def generate_random_distribution(n: int = 10000):
        """
        Generate random distribution
        :param n: number of samples
        :return: np.array of random samples
        """

        # Choose a random distribution function
        distribution_fn = np.random.choice(DistributionGenerator.distribution_fn_list)
        return distribution_fn(n)


    @staticmethod
    def generate_random_noice(n: int = 10000, limit: float = 0.05):
        """
        Generate random noice based on uniform distribution
        :param n: number of samples
        :param limit: limit of the uniform distribution
        :return: np.array of random noice
        """
        return DistributionGenerator.generate_random_uniform(n, -limit, limit * 2)

In [254]:
def create_predictors_targets_table(noise_level: float = 0.05):
    """
    Create a table with predictors and target
    :param noise_level: noise level of the target
    :return: pandas data frame with predictors and target, selected predictors and their coefficients (in tuple form)
    """

    # Generate random data
    data = {
        f"predictor_{i}": DistributionGenerator.generate_random_distribution(SAMPLE_CNT)
        for i in range(PREDICTOR_CNT)
    }

    # Choose random predictors for the target and generate coefficients for them
    all_predictors = list(data.keys())
    selected_predictors = np.random.choice(a=all_predictors, size=TARGET_PREDICTOR_CNT, replace=False)
    predictors_koefs = DistributionGenerator.generate_random_uniform(TARGET_PREDICTOR_CNT, 1.0, 2.0)

    # Generate target based on the selected predictors
    target_predictors_matrix = np.array([
        data[predictor] for predictor in selected_predictors
    ])
    target = predictors_koefs @ target_predictors_matrix

    # Add noice to the target
    noice = DistributionGenerator.generate_random_noice(SAMPLE_CNT, limit=noise_level)
    data["target"] = target + noice

    # Create a pandas data frame
    data_frame = pd.DataFrame(data)
    return data_frame, selected_predictors, predictors_koefs

In [255]:
data_frame, selected_predictors_true, predictors_koefs_true = create_predictors_targets_table()

# Separate predictors and target
X = data_frame.loc[:, data_frame.columns != 'target'].to_numpy()
y = data_frame["target"].to_numpy()

data_frame.head()

,predictor_0,predictor_1,predictor_2,predictor_3,predictor_4,predictor_5,predictor_6,predictor_7,predictor_8,predictor_9,...,predictor_991,predictor_992,predictor_993,predictor_994,predictor_995,predictor_996,predictor_997,predictor_998,predictor_999,target
0,-1.985958,0.065594,-0.846876,-4.593663,2.487855,-0.971370,0.780931,-0.255971,0.321463,-0.153328,...,0.694411,0.622125,0.918961,-4.613140,-2.948084,0.301764,-0.499084,-1.166346,1.813773,21.689586
1,0.935263,2.123331,0.480637,-0.765075,-2.103570,-1.057442,-1.031373,-0.776426,-0.464095,-1.199590,...,0.189023,1.288292,0.574721,-1.042175,-0.981104,-0.646160,-4.227038,-0.645952,1.374093,-19.015062
2,1.105202,-0.161361,0.391861,-0.346945,-0.466181,-1.403384,-0.143361,-0.092815,1.007678,-0.644908,...,-0.837571,0.937798,-0.035396,-0.089725,1.002345,-0.488152,-0.609783,-0.615746,0.218893,-4.965162
3,3.467000,-1.882663,-1.057082,1.000836,-0.863687,-1.345843,0.918456,0.085471,0.238856,-0.398822,...,-0.120372,-0.470308,0.769316,-2.296704,1.093235,-0.492854,-0.467187,-0.094619,-1.330087,-22.660378
4,2.126857,1.720015,-2.169759,-2.673064,0.339270,-0.948636,1.141215,-0.376301,3.175915,0.206297,...,-0.275861,0.119540,-0.420503,3.237286,1.065692,-0.788488,-1.717213,-0.959444,-2.929430,-18.193260


In [256]:
def show_predictors_targets_table_correlation_info(data_frame: pd.DataFrame, selected_predictors_true: np.ndarray):
    """
    Show correlation information between predictors and target
    :param data_frame: pandas data frame with predictors and target
    :param selected_predictors_true: true selected predictors
    """

    # Calculate correlation table for all the predictors and target
    correlation_table = data_frame.corr()

    # Calculate correlation information for the selected predictors for the target
    selected_correlations = correlation_table["target"][selected_predictors_true].to_numpy()
    min_selected_correlations = selected_correlations.min()

    # Calculate correlation information for the non-selected predictors for the target
    non_selected_correlations = correlation_table["target"].drop(labels=selected_predictors_true).drop(labels="target").to_numpy()
    max_non_selected_correlations = non_selected_correlations.max()

    # Calculate correlation information for the non-selected predictors for the selected predictors
    predictors_correlation_table = correlation_table.drop(columns="target").drop(labels="target")
    numpy_correlation_table = predictors_correlation_table[selected_predictors_true].drop(labels=selected_predictors_true).to_numpy()
    np.fill_diagonal(numpy_correlation_table, 0.0)
    five_most_correlated = sorted(numpy_correlation_table.flatten(), reverse=True)[:5 * 2]

    print("Minimum correlation of selected predictors - target pairs is", min_selected_correlations)
    print("Maximum correlation of non-selected predictors - target pairs is", max_non_selected_correlations)
    print("At least five pairs of non-selected predictors - selected predictors have correlation not less than", five_most_correlated[-1])

show_predictors_targets_table_correlation_info(data_frame, selected_predictors_true)

Minimum correlation of selected predictors - target pairs is -0.006135753649864581
Maximum correlation of non-selected predictors - target pairs is 0.03129463161381169
At least five pairs of non-selected predictors - selected predictors have correlation not less than 0.03687704123369397


In [257]:
def QR_decomposition(X: np.ndarray): # ~3.4s on test (1001, 1003)
    """
    Perform QR decomposition of the matrix X (self-written)
    :param X: matrix to decompose
    :return: Q and R matrices
    """

    Q = np.zeros_like(X)
    R = np.zeros((X.shape[1], X.shape[1]))

    for i in range(X.shape[1]):
        Q[:, i] = X[:, i]
        for j in range(i):
            R[j, i] = Q[:, j] @ Q[:, i]
            Q[:, i] -= R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(Q[:, i])
        Q[:, i] /= R[i, i]

    return Q, R

def QR_decomposition_fast(X: np.ndarray): # ~0.2s on test (1001, 1003)
    """
    Perform QR decomposition of the matrix X (numpy)
    :param X: matrix to decompose
    :return: Q and R matrices
    """

    return np.linalg.qr(X)

def test_QR_decomposition():
    """
    Test QR decomposition
    """

    A = np.random.rand(1001, 1003)
    Q, R = QR_decomposition(A)
    D = Q @ R - A
    if max(-D.min(), D.max()) < 1e-6:
        print("QR decomposition is correct")
    else:
        print("QR decomposition is incorrect")

test_QR_decomposition()

QR decomposition is correct


In [ ]:
class IncrementalForwardStagewiseRegression:
    def __init__(self, max_iter: int = 1_500_000, tol: float = 1e-3, step: float = 1e-2, step_decay: float = 0.8, silent: bool = True):
        """
        Initialize the class
        :param max_iter: maximum number of iterations
        :param tol: tolerance for the correlation
        :param step: step size
        :param step_decay: step decay
        :param silent
        Default values are specially selected for fit v3
        For fit v1 and v2 use max_iter: int = 15_000, tol: float = 1e-3, step: float = 1e-3
        """

        self.max_iter = max_iter
        self.tol = tol
        self.step = step
        self.step_decay = step_decay

        self.silent = silent

        self.residual = None
        self.beta = None

        self.X_mean = None
        self.X_std = None

        self.y_mean = None
        self.y_std = None

    def _fit_normalization(self, X: np.ndarray, y: np.ndarray):
        """
        Normalize data
        :param X: matrix of predictors
        :param y: target vector
        """

        self.X_mean = X.mean(axis=0)
        self.X_std = X.std(axis=0)
        self.y_mean = y.mean()
        self.y_std = y.std()

        X = (X - self.X_mean) / self.X_std
        y = (y - self.y_mean) / self.y_std

        return X, y
    
    def _normileze_X(self, X: np.ndarray):
        return (X - self.X_mean) / self.X_std
    
    def _unnormalize_y(self, y: np.ndarray):
        return y * self.y_std + self.y_mean


    def _fit_v1(self, X: np.ndarray, y: np.ndarray):
        """
        Perform incremental forward stagewise regression
        :param X: matrix of predictors
        :param y: target vector
        """

        # Normalize data
        X, y = self._fit_normalization(X, y)

        # Initialize variables
        residual = y.copy()
        self.beta = np.zeros(X.shape[1])

        for _ in range(self.max_iter):
            # Calculate correlations (slow)
            correlations = X.T @ residual
            correlations_magnitude = np.abs(correlations)

            # Find the best predictor (the one with the highest correlation)
            best_predictor = np.argmax(correlations_magnitude)
            best_correlation = correlations_magnitude[best_predictor]

            # Check if the best correlation is less than the tolerance. So the beta is predicting the target well enough
            if best_correlation < self.tol:
                print("Converged at iteration", _)
                break

            # Update beta and residual
            self.beta[best_predictor] += self.step * np.sign(correlations[best_predictor])
            residual -= self.step * np.sign(correlations[best_predictor]) * X[:, best_predictor]
            
            if not self.silent and _ != 0 and _ % 1_000 == 0:
                train_loss = (residual ** 2).mean()
                print(f"Iteration {_}, train loss: {train_loss} (MSE)")
        else:
            print("Warning: maximum number of iterations reached. Limit was", self.max_iter)

    def _fit_v2(self, X: np.ndarray, y: np.ndarray):
        X, y = self._fit_normalization(X, y)

        residual = y.copy()
        self.beta = np.zeros(X.shape[1])

        correlations = X.T @ residual

        for _ in range(self.max_iter):
            correlations_magnitude = np.abs(correlations)

            best_predictor = np.argmax(correlations_magnitude)
            best_correlation = correlations_magnitude[best_predictor]

            if best_correlation < self.tol:
                print("Converged at iteration", _)
                break

            # Update beta and correlations
            self.beta[best_predictor] += self.step * np.sign(correlations[best_predictor])
            correlations -= self.step * np.sign(correlations[best_predictor]) * (X.T @ X[:, best_predictor])

            if not self.silent:
                residual -= self.step * np.sign(correlations[best_predictor]) * X[:, best_predictor]
            
                if _ != 0 and _ % 1_000 == 0:
                    train_loss = (residual ** 2).mean()
                    print(f"Iteration {_}, train loss: {train_loss} (MSE)")
        else:
            print("Warning: maximum number of iterations reached. Limit was", self.max_iter)

    def _fit_v3(
            self,
            X: np.ndarray,
            y: np.ndarray,
        ):
        """
        Perform incremental forward stagewise regression
        (the fastest method without multiplication of the whole matrix)
        :param X: matrix of predictors
        :param y: target vector
        :param max_iter: maximum number of iterations
        :param tol: tolerance for the correlation
        :param step: step size
        :param step_decay: step decay
        """

        X, y = self._fit_normalization(X, y)

        step = self.step

        residual = y.copy()
        self.beta = np.zeros(X.shape[1])

        correlations = X.T @ residual

        XTX = X.T @ X

        for _ in range(self.max_iter):
            correlations_magnitude = np.abs(correlations)

            best_predictor = np.argmax(correlations_magnitude)
            best_correlation = correlations_magnitude[best_predictor]

            if best_correlation < self.tol:
                print("Converged at iteration", _)
                break

            # Update beta and correlations
            self.beta[best_predictor] += step * np.sign(correlations[best_predictor])
            correlations -= step * np.sign(correlations[best_predictor]) * XTX[:, best_predictor]

            if not self.silent:
                residual -= step * np.sign(correlations[best_predictor]) * X[:, best_predictor]

                if _ != 0 and _ % (self.max_iter // 100) == 0:
                    train_loss = (residual ** 2).mean()
                    print(f"Iteration {_}, train loss: {train_loss} (MSE)")

            if _ != 0 and _ % 10_000 == 0:
                step *= self.step_decay
        else:
            print("Warning: maximum number of iterations reached. Limit was", self.max_iter)
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Fit the model
        :param X: matrix of predictors
        :param y: target vector
        """

        # self._fit_v1(X, y) # ~19.7s on 15_000 iterations
        # self._fit_v2(X, y) # ~19.3s on 15_000 iterations
        self._fit_v3(X, y) # ~3.4s on 500_000 iterations (No matrix multiplication in the loop)

    def predict(self, X: np.ndarray):
        """
        Predict the target
        :param X: matrix of predictors
        :return: predicted target
        """

        if self.beta is None:
            raise ValueError("Model is not fitted")

        return self._unnormalize_y(self._normileze_X(X) @ self.beta)
    
    def compare_coefficients_with_true(self, selected_predictors_true: np.ndarray):
        """
        Compare coefficients with the true ones
        :param selected_predictors_true: true selected predictors
        """

        # Find the coefficients that are not zero
        coeffs_found = (self.beta > self.tol).sum()
        print(f"Found {coeffs_found} coefficients out of {len(selected_predictors_true)}")

        matched_coeffs = 0

        for predictor in selected_predictors_true:
            predictor_id = int(predictor.split("_")[-1])
            
            # There should be a coefficient comparator (self.beta vs predictors_koefs_true),
            # but beta was trained on normalized data and predictors_koefs_true used on non-normalized data
            # So, we can't compare them =(
            if self.beta[predictor_id] > self.tol:
                matched_coeffs += 1

        print(f"Matched {matched_coeffs} coefficients out of {len(selected_predictors_true)}")

In [259]:
model = IncrementalForwardStagewiseRegression(silent=False)
model.fit(X, y)
model.compare_coefficients_with_true(selected_predictors_true)

Iteration 15000, train loss: 0.013777021450034083 (MSE)
Iteration 30000, train loss: 0.013209539052058178 (MSE)
Iteration 45000, train loss: 0.012706747128656406 (MSE)
Iteration 60000, train loss: 0.012384810330501553 (MSE)
Iteration 75000, train loss: 0.01217562099452295 (MSE)
Iteration 90000, train loss: 0.012147906762249423 (MSE)
Iteration 105000, train loss: 0.011903439340542712 (MSE)
Iteration 120000, train loss: 0.012074409192485154 (MSE)
Iteration 135000, train loss: 0.011968114377936868 (MSE)
Iteration 150000, train loss: 0.012127480490518656 (MSE)
Iteration 165000, train loss: 0.012023185917441557 (MSE)
Iteration 180000, train loss: 0.012022784790884677 (MSE)
Iteration 195000, train loss: 0.01203851588205002 (MSE)
Iteration 210000, train loss: 0.012000156412136124 (MSE)
Iteration 225000, train loss: 0.012025201415539541 (MSE)
Iteration 240000, train loss: 0.01201633946956323 (MSE)
Iteration 255000, train loss: 0.012021059386483304 (MSE)
Iteration 270000, train loss: 0.01201137

In [260]:
def test_on_real_data(model):
    """
    Test the model on real data
    :param model: model to test
    """

    data_california = fetch_california_housing()
    X_train, X_test, y_train, y_test = train_test_split(data_california.data, data_california.target, test_size=0.3, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"MSE: {mse} | MAE: {mae}")

test_on_real_data(IncrementalForwardStagewiseRegression(max_iter=50_000))

MSE: 0.5299552900259948 | MAE: 0.52743384496775


In [261]:
test_on_real_data(LinearRegression())

MSE: 0.5305677824766757 | MAE: 0.5272474538306168


# Extra

In [346]:
def create_non_linear_predictors_targets_table(noise_level: float = 0.5):
    """
    Create a table with predictors and target with non-linear dependencies
    :param noise_level: noise level of the target
    :return: pandas data frame with predictors and target, selected predictors and their coefficients (in tuple form)
    """
    
    data = {
        f"predictor_{i}": DistributionGenerator.generate_random_uniform(SAMPLE_CNT, 0, 1)
        for i in range(PREDICTOR_CNT)
    }

    target_predictors_matrix = np.array([
        data[f"predictor_{predictor}"] for predictor in range(TARGET_PREDICTOR_CNT)
    ])
    target = np.log(target_predictors_matrix).sum(axis=0)

    # Add noice to the target
    noice = DistributionGenerator.generate_random_noice(SAMPLE_CNT, limit=noise_level)
    data["target"] = target + noice

    # Create a pandas data frame
    data_frame = pd.DataFrame(data)
    return data_frame

In [347]:
data_frame_extra = create_non_linear_predictors_targets_table()

# Separate predictors and target
X = data_frame_extra.loc[:, data_frame_extra.columns != 'target'].to_numpy()
y = data_frame_extra["target"].to_numpy()

model = IncrementalForwardStagewiseRegression(silent=False, max_iter=1_200)
model.fit(X, y)

Iteration 12, train loss: 0.9739599645160469 (MSE)
Iteration 24, train loss: 0.9524603439382854 (MSE)
Iteration 36, train loss: 0.9316979494398167 (MSE)
Iteration 48, train loss: 0.9114464109443714 (MSE)
Iteration 60, train loss: 0.8916318152396824 (MSE)
Iteration 72, train loss: 0.8722918921122684 (MSE)
Iteration 84, train loss: 0.8535305715745337 (MSE)
Iteration 96, train loss: 0.8351337779474821 (MSE)
Iteration 108, train loss: 0.8170125073070176 (MSE)
Iteration 120, train loss: 0.7992305480991873 (MSE)
Iteration 132, train loss: 0.7818274933568343 (MSE)
Iteration 144, train loss: 0.7646602738566871 (MSE)
Iteration 156, train loss: 0.7477287296011589 (MSE)
Iteration 168, train loss: 0.7311186075760736 (MSE)
Iteration 180, train loss: 0.7148927628440739 (MSE)
Iteration 192, train loss: 0.6989710015036069 (MSE)
Iteration 204, train loss: 0.6833277608701275 (MSE)
Iteration 216, train loss: 0.6679204151365903 (MSE)
Iteration 228, train loss: 0.6527582522061324 (MSE)
Iteration 240, train

Как видно, ошибка сначала падает, но затем начинает расти. Падение происходит в силу малых аппроксимаций большим количеством равномерных распределений [-0.5, 0.5] (не забываем про нормировку), близких к 0 значений. Но когда начинают оставаться большие значений, регрессия не может апроксимировать такие сложные функции как `log` линейными, из-за чего начнинает просто добавлять шум.